<a href="https://colab.research.google.com/github/Zardian18/Harry-porter-story-generation/blob/master/LSTM_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/Zardian18/helper-functions-colab/master/helper.py

--2024-01-25 09:13:54--  https://raw.githubusercontent.com/Zardian18/helper-functions-colab/master/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17171 (17K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]  16.77K  --.-KB/s    in 0s      

2024-01-25 09:13:54 (84.7 MB/s) - ‘helper.py’ saved [17171/17171]



In [2]:
!pip install kaggle

In [3]:
from google.colab import drive
drive.mount("/content/drive")

! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [4]:
import numpy as np
import json
import re
import string
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses
import pandas as pd

In [5]:
! kaggle datasets download -d utsavk02/4-light-novel-for-text-generation

 83% 3.00M/3.61M [00:00<00:00, 5.58MB/s]
100% 3.61M/3.61M [00:00<00:00, 5.54MB/s]


In [6]:
! unzip 4-light-novel-for-text-generation.zip

Archive:  4-light-novel-for-text-generation.zip
  inflating: Against the Gods/Against the Gods - Volume 1.txt  
  inflating: Level 999 Villager/LV999-Villager-Volume-1.txt  
  inflating: Level 999 Villager/LV999-Villager-Volume-2.txt  
  inflating: Level 999 Villager/LV999-Villager-Volume-3.txt  
  inflating: Level 999 Villager/LV999-Villager-Volume-4.txt  
  inflating: Level 999 Villager/LV999-Villager-Volume-5.txt  
  inflating: Level 999 Villager/LV999-Villager-Volume-6.txt  
  inflating: The Dungeon Seeker/The-Dungeon-Seeker-Volume-3.txt  
  inflating: The Dungeon Seeker/The-Dungeon-Seeker-Volume-4.txt  
  inflating: The Skeleton Knight/Skeleton Knight - Volume 4.txt  
  inflating: The Skeleton Knight/Skeleton Knight - Volume 5.txt  
  inflating: The Skeleton Knight/Skeleton Knight - Volume 6.txt  
  inflating: harrypotter/Book 1 - The Philosophers Stone.txt  
  inflating: harrypotter/Book 2 - The Chamber of Secrets.txt  
  inflating: harrypotter/Book 3 - The Prisoner of Azkaban.tx

In [7]:
file_paths = ['/content/harrypotter/Book 1 - The Philosophers Stone.txt',
              '/content/harrypotter/Book 2 - The Chamber of Secrets.txt',
              '/content/harrypotter/Book 3 - The Prisoner of Azkaban.txt',
              '/content/harrypotter/Book 4 - The Goblet of Fire.txt',
              '/content/harrypotter/Book 5 - The Order of the Phoenix.txt',
              '/content/harrypotter/Book 6 - The Half Blood Prince.txt',
              '/content/harrypotter/Book 7 - The Deathly Hallows.txt']

all_paragraphs = []
for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    paragraphs = content.split('\n\n')
    all_paragraphs.extend(paragraphs)

harrypotter_df = pd.DataFrame({'Conversations': all_paragraphs})

In [8]:
harrypotter_df

,Conversations
0,/
1,
2,\nTHE BOY WHO LIVED
3,"Mr. and Mrs. Dursley, of number four, Privet D..."
4,Mr. Dursley was the director of a firm called ...
...,...
54486,“I know he will.”
54487,The scar had not pained Harry for nineteen yea...
54488,Page | 856 Harry Potter and the Deathly Hallow...
54489,


In [10]:
harrypotter_df.isna().sum()

Conversations    0
dtype: int64

In [15]:
harrypotter_df["Conversations"][4]

'Mr. Dursley was the director of a firm called \nGrunnings, which made drills. He was a big, beefy \nman with hardly any neck, although he did have a \nvery large mustache. Mrs. Dursley was thin and \nblonde and had nearly twice the usual amount of \nneck, which came in very useful as she spent so \nmuch of her time craning over garden fences, spying \non the neighbors. The Dursley s had a small son \ncalled Dudley and in their opinion there was no finer \nboy anywhere. '

In [18]:
df_filtered = harrypotter_df[harrypotter_df['Conversations'].apply(lambda x: len(x.split()) >= 10)]

In [25]:
df_filtered_reset = df_filtered.reset_index(drop=True)

In [27]:
df_filtered_reset["Conversations"][0]

'Mr. and Mrs. Dursley, of number four, Privet Drive, \nwere proud to say that they were perfectly normal, \nthank you very much. They were the last people you’d \nexpect to be involved in anything strange or \nmysterious, because they just didn’t hold with such \nnonsense. '

In [28]:
len(df_filtered_reset)

36257

In [29]:
df_filtered_reset.head(5)

,Conversations
0,"Mr. and Mrs. Dursley, of number four, Privet D..."
1,Mr. Dursley was the director of a firm called ...
2,"The Dursleys had everything they wanted, but t..."
3,Page | 2 Harry Potter and the Philosophers Sto...
4,"met for several years; in fact, Mrs. Dursley p..."


In [31]:
para_list=[]

for i in range(len(df_filtered_reset)):
  para_list.append(df_filtered_reset["Conversations"][i])


para_list[:5]

['Mr. and Mrs. Dursley, of number four, Privet Drive, \nwere proud to say that they were perfectly normal, \nthank you very much. They were the last people you’d \nexpect to be involved in anything strange or \nmysterious, because they just didn’t hold with such \nnonsense. ',
 'Mr. Dursley was the director of a firm called \nGrunnings, which made drills. He was a big, beefy \nman with hardly any neck, although he did have a \nvery large mustache. Mrs. Dursley was thin and \nblonde and had nearly twice the usual amount of \nneck, which came in very useful as she spent so \nmuch of her time craning over garden fences, spying \non the neighbors. The Dursley s had a small son \ncalled Dudley and in their opinion there was no finer \nboy anywhere. ',
 'The Dursleys had everything they wanted, but they \nalso had a secret, and their greatest fear was that \nsomebody would discover it. They didn’t think they \ncould bear it if anyone found out about the Potters. \nMrs. Potter was Mrs. Dursle

In [32]:
para_list[0]

'Mr. and Mrs. Dursley, of number four, Privet Drive, \nwere proud to say that they were perfectly normal, \nthank you very much. They were the last people you’d \nexpect to be involved in anything strange or \nmysterious, because they just didn’t hold with such \nnonsense. '

In [33]:
def pad_punctuation(s):
  s = re.sub(f"([{string.punctuation}])", r' \1', s)
  s = re.sub(" +", " ", s)
  return s

In [34]:
text_data = [pad_punctuation(x) for x in para_list]

In [35]:
text_data[0]

'Mr . and Mrs . Dursley , of number four , Privet Drive , \nwere proud to say that they were perfectly normal , \nthank you very much . They were the last people you’d \nexpect to be involved in anything strange or \nmysterious , because they just didn’t hold with such \nnonsense . '

In [36]:
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(32).shuffle(1000)

In [37]:
text_ds

<_ShuffleDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [39]:
sum =0
for i in range(len(text_data)):
  words = text_data[i].split()
  sum += len(words)

print(f"Average words per Conversation: {sum/len(text_data)}")

Average words per Conversation: 35.93504702540199


In [40]:
vectorize_layer = layers.TextVectorization(
    standardize= "lower",
    max_tokens = 10000,
    output_mode = "int",
    output_sequence_length = 50,
)

In [41]:
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [42]:
vocab[:10]

['', '[UNK]', '.', ',', 'the', 'and', 'to', 'of', 'a', 'he']

In [43]:
vocab[-10:]

['“tom',
 '“they’d',
 '“there’ll',
 '“their',
 '“that’d',
 '“talking',
 '“such',
 '“sturgis',
 '“stupid',
 '“stuff']

In [44]:
try:
    index = vocab.index("\n")
    print(f"'\n' found at index {index}")
except ValueError:
    print("'\n' not found in vocab")

'
' not found in vocab


In [45]:
vectorize_layer((text_data[0]))

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([ 149,    2,    5,  199,    2, 1440,    3,    7,  683,  497,    3,
       1361, 1078,    3,   46, 2110,    6,  196,   19,   29,   46, 1251,
       1151,    3, 1675,   16,   91,  164,    2,   29,   46,    4,  160,
        184,  586,  845,    6,   43, 2594,   13,  223, 1015,  108, 2686,
          3,  209,   29,   86,  134,  829])>

In [47]:
text_data[0]

'Mr . and Mrs . Dursley , of number four , Privet Drive , \nwere proud to say that they were perfectly normal , \nthank you very much . They were the last people you’d \nexpect to be involved in anything strange or \nmysterious , because they just didn’t hold with such \nnonsense . '

In [48]:
def prepare_inputs(text):
  text = tf.expand_dims(text, axis=-1)
  tokenized_sentences = vectorize_layer(text)
  x = tokenized_sentences[:,:-1]
  y = tokenized_sentences[:,1:]

  return x,y

In [49]:
train_ds = text_ds.map(prepare_inputs)

In [50]:
train_ds

<_MapDataset element_spec=(TensorSpec(shape=(None, 49), dtype=tf.int64, name=None), TensorSpec(shape=(None, 49), dtype=tf.int64, name=None))>

In [52]:
inputs = layers.Input(shape=(None, ), dtype="int32")
x = layers.Embedding(10000, 150)(inputs)
x = layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(x)

outputs = layers.Dense(10000, activation="softmax")(x)

lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 150)         1500000   
                                                                 
 lstm_2 (LSTM)               (None, None, 128)         142848    
                                                                 
 bidirectional_1 (Bidirecti  (None, None, 128)         98816     
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, None, 10000)       1290000   
                                                                 
Total params: 3031664 (11.56 MB)
Trainable params: 3031664 (11.56 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [53]:
lstm.compile(loss= losses.SparseCategoricalCrossentropy(),
             optimizer="adam")

In [54]:
import random

In [55]:
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:  # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        # wordx = np.random.randint(1, 10000)
        # vocabx= vocab[wordx]
        selected_words = random.sample(vocab, 10)

        # Join the selected words into a single string
        result_string = ' '.join(selected_words)
        self.generate(f"{result_string}", max_tokens=49, temperature=1.0)

In [56]:
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

text_generator = TextGenerator(vocab)

In [58]:
lstm.fit(
    train_ds,
    epochs=25,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/25
1134/1134 [==============================] - ETA: 0s - loss: 3.8650
generated text:
sheets comet repair ’is soaked lock affection darkly funny languages fudge to go to thought , defense tightly ,” 

1134/1134 [==============================] - 563s 486ms/step - loss: 3.8650
Epoch 2/25
1134/1134 [==============================] - ETA: 0s - loss: 3.0930
generated text:
strutting reprovingly gaunts charged grandparents junk saved lightly unnerved pajamas for a middle of when his night prickled will deafening collapsed in usual and mind with his with desperate alarmed and them fears turned , and she didn’t people heard out again mungo’s through a fingers of peering into

1134/1134 [==============================] - 521s 459ms/step - loss: 3.0930
Epoch 3/25
1134/1134 [==============================] - ETA: 0s - loss: 2.5901
generated text:
belief prophecy rubbish effective distant amidst stored bows resort fussing considering you’ve can’t your time durmstrang book . we’ve spring 

KeyboardInterrupt: 

In [59]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [64]:
info = text_generator.generate(
    "voldemort", max_tokens=150, temperature=1
)


generated text:
voldemort could have seen to tell using elder clutches as a new [UNK] , all still humming . piers , and but the thought top , harry will share with [UNK] , a third wizard likely this horrible [UNK] agreed to join in the tournament . he wanted to him who ’ve uses the news from him to want to suffer . got scabbers’s to escorted to notice it covered and there , they naturally had come , [UNK] occurred to the cell and weekends into the river high to page had appeared to side a [UNK] with a twitch of it over the team in the creature’s and [UNK] of his arms , and marmalade them . 



In [65]:
print_probs(info, vocab)


PROMPT: voldemort
was:   	22.67%
had:   	10.34%
,:   	10.13%
and:   	3.63%
were:   	3.61%
--------


PROMPT: voldemort could
have:   	8.24%
be:   	7.79%
make:   	4.5%
feel:   	4.41%
see:   	4.34%
--------


PROMPT: voldemort could have
been:   	19.09%
a:   	6.19%
to:   	5.23%
the:   	3.1%
misunderstood:   	2.0%
--------


PROMPT: voldemort could have seen
to:   	13.52%
the:   	10.45%
,:   	7.6%
that:   	5.29%
a:   	2.78%
--------


PROMPT: voldemort could have seen to
be:   	6.68%
make:   	3.05%
[UNK]:   	2.42%
distract:   	2.27%
suffer:   	2.19%
--------


PROMPT: voldemort could have seen to tell
the:   	11.84%
whether:   	11.3%
that:   	9.63%
him:   	6.26%
to:   	5.36%
--------


PROMPT: voldemort could have seen to tell using
the:   	23.85%
his:   	5.4%
a:   	3.65%
it:   	3.01%
harry’s:   	2.13%
--------


PROMPT: voldemort could have seen to tell using elder
decision:   	6.27%
suspicions:   	4.31%
best:   	3.07%
sanity:   	2.3%
feet:   	2.29%
--------


PROMPT: voldemort could ha

In [62]:
def generate_text_from_model(model, index_to_word, start_prompt, max_tokens=100, temperature=1.0):
    word_to_index = {word: index for index, word in enumerate(index_to_word)}

    def sample_from(probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    start_tokens = [word_to_index.get(x, 1) for x in start_prompt.split()]
    sample_token = None
    generated_text = ""

    while len(start_tokens) < max_tokens and sample_token != 0:
        x = np.array([start_tokens])
        y = model.predict(x, verbose=0)
        sample_token, _ = sample_from(y[0][-1], temperature)
        start_tokens.append(sample_token)
        generated_text += " " + index_to_word[sample_token]

    return generated_text.strip()

model = lstm
index_to_word = vocab

In [67]:
start_prompt = "harry saw voldemort and"

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=1)
print(f"Generated text:\n{start_prompt + generated_text}")

Generated text:
harry saw voldemort and  scratching him once and calling him barking . his footsteps fell over over for medieval occasions . they had begun to defend the passageway ; cho’s eyebrows leapt wordlessly upward , the reductor locket fly onto their room into the pages and the spells , of


In [68]:
start_prompt = "harry saw voldemort and "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=1)
print(f"Generated text:\n{start_prompt + generated_text}")

Generated text:
harry saw voldemort andsuddenly pointing to a hand with his mug who fell off it , and leaned past the veela , toward her will small raspberry . the graves fell to plant bumped on the same rain gilded across before the brim bared the way in a heap


In [69]:
start_prompt = "hermoine wanted "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=1)
print(f"Generated text:\n{start_prompt + generated_text}")

Generated text:
hermoine wanted you to be lying and have finished ?” ron shouted in a somber whisper , a cheerful profile . “dumbledore ’as him not an [UNK] idea in an next dementor ! he’s big of victims , the diary fell — ”


In [73]:
start_prompt = "hermoine wanted "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=1)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

hermoine wanted 


'to have a plan of it this , a married goblin has ?” he breathed , pausing , clutching him hopelessly . “winky , ron , and you let you mean if we’re insulting heights — ”'

In [74]:
start_prompt = "hermoine wanted "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=0.75)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

hermoine wanted 


'at a third sacrifice , i’ll be to no one of [UNK] , which will be being able to be crushed from you if he was fit a recovery we’ll accept the delay at the bishop .'

In [78]:
info = text_generator.generate(
    "hermoine wants", max_tokens=150, temperature=1
)


generated text:
hermoine wants the charm that arguments straight the shop fruit , the shop tower . rowling 



In [79]:
print_probs(info, vocab)


PROMPT: hermoine wants
to:   	18.09%
the:   	12.89%
,:   	9.44%
a:   	5.91%
that:   	3.34%
--------


PROMPT: hermoine wants the
[UNK]:   	4.16%
way:   	2.83%
elder:   	2.0%
time:   	1.67%
thing:   	1.5%
--------


PROMPT: hermoine wants the charm
,:   	19.52%
to:   	14.87%
in:   	11.66%
of:   	5.68%
on:   	5.56%
--------


PROMPT: hermoine wants the charm that
you:   	6.49%
the:   	5.26%
he:   	3.49%
it:   	2.93%
they:   	2.86%
--------


PROMPT: hermoine wants the charm that arguments
,:   	30.88%
?”:   	16.73%
!”:   	7.36%
.:   	4.18%
and:   	3.28%
--------


PROMPT: hermoine wants the charm that arguments straight
past:   	10.33%
,:   	9.11%
right:   	5.84%
over:   	5.34%
to:   	4.56%
--------


PROMPT: hermoine wants the charm that arguments straight the
quaffle:   	4.94%
[UNK]:   	4.7%
grounds:   	3.44%
stairs:   	2.6%
field:   	1.53%
--------


PROMPT: hermoine wants the charm that arguments straight the shop
in:   	10.96%
,:   	7.44%
[UNK]:   	5.71%
.:   	4.71%
of:   	3.81%
--

In [82]:
start_prompt = "Harry rushed towrads Voldemort and witnessed "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=1)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Harry rushed towrads Voldemort and witnessed 


'them , looking out -of -bounds when anyone would feel an appearance that nothing .'

In [84]:
start_prompt = "Harry rushed towrads Voldemort and witnessed "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=1)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Harry rushed towrads Voldemort and witnessed 


'the scene in the bag , and rows itself toward smoothly witches and blood again , heavily draped with the rack sore new [UNK] , but the water seemed berserk on the shop and wilder [UNK] . though he grabbed a sickening slope to'

In [86]:
start_prompt = "Voldemort's son joined hogwards but "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=1)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Voldemort's son joined hogwards but 


'it’s what you sought harm to go [UNK] , when delay them are the events . reality feels to ask merope tonight so about him as he survives the heads , what’s to create . we made contact us about to extract the ministry of'

In [89]:
start_prompt = "Voldemort's son joined hogwards but "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=150, temperature=0.75)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Voldemort's son joined hogwards but 


'they will consent work , to this discussion of the world . . . .”'

In [90]:
start_prompt = "Even though harry destroyed the elder wand, it was then rebuilt by "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=150, temperature=0.75)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Even though harry destroyed the elder wand, it was then rebuilt by 


'the meantime , from the wall twice so [UNK] [UNK] , which were not supposed to be sure it was nowhere properly on , he was just amazed .'

In [91]:
start_prompt = "Even though harry destroyed the elder wand, it was then rebuilt by "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=150, temperature=0.8)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Even though harry destroyed the elder wand, it was then rebuilt by 


'one experts , which i only have wizard managed to [UNK] all to potter , what would be always this out of retirement that the inquisitorial squad , it , whether you sees contact in the end of grindelwald , which is not necessary on the point , could ignore my toad , you think to accept me [UNK] [UNK] things , he was regularly , and madam pomfrey made more reasons you as you -know -who -must .w room was hurting to find it , you see anyone , but it passed to serve the end of [UNK] cunning as he had taken him with mummy , took the dursleys , you make more rare wizarding maladies than lessons to extract as august and monsters , neither ! what concerns , it , he was banished to'

In [99]:
start_prompt = "Sirius "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=0.85)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Sirius 


', the committee of ’em , which does not forgive him , we were all training , and you will be trapped to him on the train'

In [112]:
start_prompt = random.sample(vocab, 2)
start_prompt = ' '.join(start_prompt)
generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=0.85)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

ending repeated


'them at the top of [UNK] in the [UNK] halloween , though you cannot bear be able to suffer .”'

In [117]:
start_prompt = random.sample(vocab, 1)
start_prompt = ' '.join(start_prompt)
generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=0.85)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

puffy


'underwater and hermione , ron , ron , whom tracing her gaze slid into the front of this way to the trunks in front of the hoop . it was him , voldemort’s dear as usual than make the tiniest longer move out to the counter , the wall'

In [122]:
start_prompt = "Elder wand was "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=1)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Elder wand was 


'leaning past the time top before steps on the classroom passed . the night were using [UNK] , the keep present or him went and the [UNK] fridge , it and derrick .'

In [127]:
start_prompt = "Snape "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=1)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Snape 


'pardon bode and a friend of deliberation . this more [UNK] of july was becoming -blood prince snare , whose appearance was supposed to be sending it this .l .s had been [UNK] from voldemort’s middle -ended fingernails , once a walk gillyweed and myrtle fell up through to'

In [128]:
start_prompt = "After the events of hogwards "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=500, temperature=1)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

After the events of hogwards 


', won’t mention harry’s time , would forget about my palace , a cry finished , we -know -who -must -be -named to think , countless us , be when you mean , it [UNK] , he knows he’s allowed him on . goyle , i’ll [UNK] habit , it would have foreseen , they must dispose give a little a kind of wizards’ [UNK] room , the ball poltergeist in king’s to safely into the confederation of the kitchens , gornuk had told this joke , which scar was really joke , will miss a campaign , will not owls'

In [135]:
start_prompt = "After the events of hogwards "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=500, temperature=0.9)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

After the events of hogwards 


'molly , firenze will not bring a different period about all beside the cellar or you was slaughtered to your inquisitorial squad .'

In [140]:
start_prompt = "Hocrux "

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=0.9)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Hocrux 


', which was not really reappearing if [UNK] about the inquisitorial squad together . . omens , that’s the reason most as he was feeling very to wizard . . . but we won’t send us it’s not performing weeks about unstable , and it was no good of'

In [153]:
start_prompt = "Dumbledore fought"

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=0.9)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Dumbledore fought


'the president of the rest of veela , dobby were elder dad up me , we were doing to allow me , or [UNK] me to be buying the thing , be in the rubble across a third time this ! i dropped to your ownership , with'

In [155]:
start_prompt = "The strongest wizard"

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=50, temperature=0.9)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

The strongest wizard


', harry , the [UNK] steps to king’s look to a run . “are a [UNK] [UNK] wizard just recognized the ministry and delicate - exhaust murders is genially up on the covers behind a shop friend , well , if whoever was guarding its subject —'

## Generating a meaningful story

In [157]:
start_prompt = "Harry saw a suspecious man"

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=10, temperature=0.9)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

Harry saw a suspecious man


'as they soared onto one . “jus’ without all [UNK] . bleedin’ blood , neither you don’t seem more sure that we wait me . “karkaroff can’t accept the [UNK] is allowed out to spend us — ”'

In [160]:
start_prompt = "So harry followed him and found out"

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=30, temperature=0.9)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

So harry followed him and found out


'of rubble on the bag a lawns between her hands very startled . “winky was throwing a fight before you see there’s sirius'

In [163]:
start_prompt = "sirius joined harry in the fight against"

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=30, temperature=0.9)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

sirius joined harry in the fight against


'the bottom of under , and burst as the smell of one -armed appeared she had dizzy . “master had have been given'

In [164]:
start_prompt = "now he was fighting alone"

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=30, temperature=0.9)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

now he was fighting alone


'to further journey to enemies ; how only him , on the tower himself , as he was a moments and allowed the halloween little'

In [168]:
start_prompt = "he took out his wand and"

generated_text = generate_text_from_model(model, index_to_word, start_prompt, max_tokens=30, temperature=0.9)
# print(f"Generated text:\n{start_prompt + generated_text}")
print(start_prompt)
generated_text

he took out his wand and


'ron , it fell backward onto the wreckage to a palace , harry , and the ideal combination of whom was wasting to [UNK]'